# Couchbase
> Couchbase 是一个屡获殊荣的分布式 NoSQL 云数据库，为您的所有云、移动、AI 和边缘计算应用程序提供无与伦比的多功能性、性能、可伸缩性和财务价值。Couchbase 通过为开发人员提供编码辅助和为其应用程序提供向量搜索来拥抱 AI。

本笔记本将介绍如何使用 `CouchbaseChatMessageHistory` 类将聊天消息历史记录存储在 Couchbase 集群中。

## 设置 Couchbase 集群
要运行此演示，您需要一个 Couchbase 集群。

您可以同时使用 [Couchbase Capella](https://www.couchbase.com/products/capella/) 和您自管理的 Couchbase Server。

## 安装依赖
`CouchbaseChatMessageHistory` 位于 `langchain-couchbase` 包中。

In [1]:
%pip install --upgrade --quiet langchain-couchbase

Note: you may need to restart the kernel to use updated packages.


## 创建 Couchbase 连接对象
我们首先创建到 Couchbase 集群的连接，然后将该集群对象传递给 Vector Store。

这里我们使用用户名和密码进行连接。您也可以使用其他任何支持的方式连接到您的集群。

有关连接到 Couchbase 集群的更多信息，请参阅 [Python SDK 文档](https://docs.couchbase.com/python-sdk/current/hello-world/start-using-sdk.html#connect)。

In [2]:
COUCHBASE_CONNECTION_STRING = (
    "couchbase://localhost"  # or "couchbases://localhost" if using TLS
)
DB_USERNAME = "Administrator"
DB_PASSWORD = "Password"

In [3]:
from datetime import timedelta

from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions

auth = PasswordAuthenticator(DB_USERNAME, DB_PASSWORD)
options = ClusterOptions(auth)
cluster = Cluster(COUCHBASE_CONNECTION_STRING, options)

# Wait until the cluster is ready for use.
cluster.wait_until_ready(timedelta(seconds=5))

我们现在将在您的 Couchbase 集群中设置您想要用于存储消息历史记录的 bucket、scope 和 collection 名称。

请注意，在存储消息历史记录之前，bucket、scope 和 collection 需要已经存在。

In [4]:
BUCKET_NAME = "langchain-testing"
SCOPE_NAME = "_default"
COLLECTION_NAME = "conversational_cache"

## 使用方法
为了存储消息，您需要以下内容：
- Couchbase Cluster 对象：与 Couchbase 集群的有效连接
- bucket_name：集群中用于存储聊天消息历史的存储桶
- scope_name：存储桶中用于存储消息历史的范围
- collection_name：范围中用于存储消息历史的集合
- session_id：会话的唯一标识符

您还可以配置以下可选项目：
- session_id_key：在聊天消息文档中用于存储 `session_id` 的字段
- message_key：在聊天消息文档中用于存储消息内容的字段
- create_index：用于指定是否需要在集合上创建索引。默认情况下，会在文档的 `message_key` 和 `session_id_key` 上创建索引
- ttl：用于指定文档的生存时间（以 `timedelta` 表示），之后它们将自动从存储中删除。

In [5]:
from langchain_couchbase.chat_message_histories import CouchbaseChatMessageHistory

message_history = CouchbaseChatMessageHistory(
    cluster=cluster,
    bucket_name=BUCKET_NAME,
    scope_name=SCOPE_NAME,
    collection_name=COLLECTION_NAME,
    session_id="test-session",
)

message_history.add_user_message("hi!")

message_history.add_ai_message("how are you doing?")

In [6]:
message_history.messages

[HumanMessage(content='hi!'), AIMessage(content='how are you doing?')]

## 指定聊天消息的生存时间（TTL）
通过在初始化聊天消息历史记录存储时指定 `ttl` 参数，可以自动在指定的未来时间点删除已存储的消息。

In [7]:
from langchain_couchbase.chat_message_histories import CouchbaseChatMessageHistory

message_history = CouchbaseChatMessageHistory(
    cluster=cluster,
    bucket_name=BUCKET_NAME,
    scope_name=SCOPE_NAME,
    collection_name=COLLECTION_NAME,
    session_id="test-session",
    ttl=timedelta(hours=24),
)

## 链式调用
聊天记录类可以与 [LCEL 可运行对象](https://python.langchain.com/docs/how_to/message_history/) 结合使用

In [8]:
import getpass
import os

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

# Create the LCEL runnable
chain = prompt | ChatOpenAI()

In [10]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: CouchbaseChatMessageHistory(
        cluster=cluster,
        bucket_name=BUCKET_NAME,
        scope_name=SCOPE_NAME,
        collection_name=COLLECTION_NAME,
        session_id=session_id,
    ),
    input_messages_key="question",
    history_messages_key="history",
)

In [11]:
# This is where we configure the session id
config = {"configurable": {"session_id": "testing"}}

In [12]:
chain_with_history.invoke({"question": "Hi! I'm bob"}, config=config)

AIMessage(content='Hello, Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 22, 'total_tokens': 33}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-62e54e3d-db70-429d-9ee0-e5e8eb2489a1-0', usage_metadata={'input_tokens': 22, 'output_tokens': 11, 'total_tokens': 33})

In [13]:
chain_with_history.invoke({"question": "Whats my name"}, config=config)

AIMessage(content='Your name is Bob.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 44, 'total_tokens': 49}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d84a570a-45f3-4931-814a-078761170bca-0', usage_metadata={'input_tokens': 44, 'output_tokens': 5, 'total_tokens': 49})